## Imports

In [99]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [100]:
df = pd.read_csv('redfin_2023-05-26-10-05-04.csv')

## Data Clean

In [101]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [102]:
print(len(df))
df = df.dropna(subset=['SOLD DATE'])
print('-----')
print(len(df))

7592
-----
6778


In [103]:
# Desired month
desired_month = 'April'

# Define list of desired months (excluding current month)
desired_months = [desired_month]

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [104]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    2265
Name: count, dtype: int64
-------
$/SQUARE FEET
False    2071
True      194
Name: count, dtype: int64
-------
YEAR BUILT
False    2179
True       86
Name: count, dtype: int64
-------


In [105]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [106]:
# df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
# df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
# df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
# df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
# df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [107]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
309,PAST SALE,April-21-2023,Single Family Residential,3160 223rd St,Sauk Village,IL,60411.0,15000.0,1.0,NaN,...,Sold,NaN,NaN,https://www.redfin.com/IL/Sauk-Village/3160-22...,MRED,11745532.0,N,Y,41.484411,-87.543734
761,PAST SALE,April-19-2023,Single Family Residential,555 E 92nd Pl,Chicago,IL,60619.0,26500.0,3.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/555-E-92nd-P...,MRED,11738065.0,N,Y,41.726331,-87.609692
558,PAST SALE,April-10-2023,Single Family Residential,22625 Theodore Ave,Sauk Village,IL,60411.0,27000.0,4.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Sauk-Village/22625-T...,MRED,11737200.0,N,Y,41.479543,-87.549532
2224,PAST SALE,April-4-2023,Single Family Residential,211 155th St,Calumet City,IL,60409.0,29000.0,3.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Calumet-City/211-155...,MRED,11608695.0,N,Y,41.613665,-87.530525
1438,PAST SALE,April-12-2023,Single Family Residential,14523 S Justine St,Harvey,IL,60426.0,30000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Harvey/14523-Justine...,MRED,11698681.0,N,Y,41.625369,-87.654537
325,PAST SALE,April-26-2023,Single Family Residential,6539 S Aberdeen St,Chicago,IL,60621.0,31000.0,3.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/6539-S-Aberd...,MRED,11757223.0,N,Y,41.774858,-87.651685
655,PAST SALE,April-14-2023,Single Family Residential,6521 S Laflin St,Chicago,IL,60636.0,33400.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/6521-S-Lafli...,MRED,11738589.0,N,Y,41.775233,-87.661401
379,PAST SALE,April-24-2023,Single Family Residential,6752 S Aberdeen St,Chicago,IL,60621.0,35000.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/6752-S-Aberd...,MRED,11713451.0,N,Y,41.770819,-87.652027
1868,PAST SALE,April-4-2023,Single Family Residential,7442 S Princeton Ave,Chicago,IL,60621.0,35000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/7442-S-Princ...,MRED,11464161.0,N,Y,41.758675,-87.632345
1685,PAST SALE,April-12-2023,Single Family Residential,14615 Winchester Ave,Harvey,IL,60426.0,35200.0,4.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/IL/Harvey/14615-Winches...,MRED,11684334.0,N,Y,41.623857,-87.664195


In [108]:
print(df_filtered['URL'].iloc[416])

https://www.redfin.com/IL/Chicago/1710-N-Crilly-Ct-60614/home/13344202


In [109]:
# # Correct the prices, if needed
# df_filtered.at[618,'PRICE']=(295000)
# df_filtered.at[832,'PRICE']=(355000)

In [110]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [111]:
# # Corrections, if needed
# df_filtered.at[1279,'$/SQUARE FEET']=(395000/1316)
# df_filtered.at[402,'$/SQUARE FEET']=(220000/1244)

In [112]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['ADDRESS','CITY','$/SQUARE FEET']]

,ADDRESS,CITY,$/SQUARE FEET
1474,3638 Arthur Ter,Markham,4.0
309,3160 223rd St,Sauk Village,11.0
2046,154 E 148th St,Harvey,12.0
2224,211 155th St,Calumet City,15.0
805,5145 S Fairfield Ave,Chicago,18.0
1890,6028 S Winchester Ave,Chicago,18.0
784,12148 S Lowe Ave,Chicago,19.0
604,16609 Kenwood Ave,South Holland,20.0
558,22625 Theodore Ave,Sauk Village,21.0
323,8636 S Lowe Ave,Chicago,23.0


In [113]:
print(df_filtered.URL.iloc[309])

https://www.redfin.com/IL/Sauk-Village/3160-223rd-St-60411/home/12818190


In [114]:
# Corrections, if needed
#
#
#

In [115]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [116]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [117]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [118]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [119]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [120]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [121]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"{desired_month} 2023 Single Family Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [122]:
m.save('index.html')

## Summary Info

In [123]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [124]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [125]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [126]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
CHI - Near North Side, 1401 N Dearborn St Chicago | Price $5,650,000 | $653 psf | Year built: 1877
Least Expensive
Sauk Village, 3160 223rd St Sauk Village | Price $15,000 | $11 psf | Year built: 1979
Highest Price Per Square Foot
CHI - Lincoln Park, 453 W Deming Pl Chicago | Price $4,500,000 | $858 psf | Year built: 2015
Lowest Price Per Square Foot
Mcintosh, 3638 Arthur Ter Markham | Price $63,000 | $4 psf | Year built: 1947
Newest
CHI - North Center, 1934 W Patterson Ave Chicago | Price $2,275,000 | $nan psf | Year built: 2023
Oldest
Dolton, 14717 Dr. Martin Luther King Jr. Dr Dolton | Price $170,000 | $94 psf | Year built: 1808


## Time on Market Calculator

In [127]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [128]:
# from datetime import datetime, timedelta

# date1 = datetime(2022, 12, 15) ## List (Earlier) date
# date2 = datetime(2023, 3, 30) ## Close (Later) date

# delta = date2 - date1
# num_days = delta.days

# print(num_days)

## Map URL Snagger

In [129]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [130]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CookCounty_SingleFamily_sales_month_ending_april_2023


## Get Summary Data

In [131]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 2265
--------
Total sale price: $962,187,424
--------
Median sale price: $329,000
--------
Max sale price: $5,650,000
--------
Min sale price: $15,000
------------------------------------------------
PSF INFO
Max price per square foot: $858
--------
Min price per square foot: $4
--------
Median price per square foot: $200
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1808.0
----------
Average building age: 1953.8903166590178


## To CSV

In [132]:
df_filtered.to_csv(f'{desired_month}_SingleFamily.csv')